In [1]:
import torch
from dptb.nnops.nnapi import NNSK
from dptb.structure.structure import BaseStruct
from dptb.dataprocess.datareader import read_data
import matplotlib.pyplot as plt
import pickle as pickle
import numpy as np


if __name__ == '__main__':
    checkpointPath = "/root/dptb_exp/silicon/checkpoint/best_nnsk.pth"
    train_data_path = "/root/dptb_exp/silicon/data/"
    train_data_prefix = "set"

    proj_atom_neles = {"Si": 4}
    proj_anglr_m = {
      "Si": [
        "3s",
        "3p"
      ]
    }
    nnsk = NNSK(checkpointPath, proj_anglr_m)
    band = pickle.load(open('/root/dptb_exp/silicon/data/band_structure.pickle', 'rb'))

    filename = '/root/dptb_exp/silicon/data/silicon.vasp'
    st = BaseStruct(atom=filename, format='vasp',
                    cutoff=4.0, proj_atom_anglr_m=proj_anglr_m, proj_atom_neles=proj_atom_neles, onsitemode='uniform', onsite_strain=True)

    snapase = st.struct
    # snapase = structase
    lat = snapase.cell.get_bravais_lattice()
    print(lat.description())
    # lat.plot_bz(show=True)
    special_kp = lat.get_special_points()
    # spmap['M'] = np.array([0.5,0.5,1])
    # kpath=snapase.cell.bandpath('XGX', npoints=120)
    kpath = snapase.cell.bandpath('GXUKGLWX', npoints=400)
    # kpath = snapase.cell.bandpath('WLGXWK', npoints=400)
    xlist, high_sym_kpoints, labels = kpath.get_linear_kpoint_axis()
    klist = kpath.kpts
    
    all_bonds, hamil_blocks, overlap_blocks = nnsk.get_HR(st)
    hkmat, skmat = nnsk.get_HK([klist[0]])

    eigks, EF = nnsk.get_eigenvalues(klist)

    plt.figure(figsize=(5, 5), dpi=100)
    for i in range(eigks.shape[1]):
        plt.plot(xlist, eigks[:, i] - np.min(eigks), 'r-', lw=1)

    for ib in range(band['eigenvalues'].shape[1]):
        plt.plot(band['xcoords'], band['eigenvalues'][:, ib] - np.min(band['eigenvalues']), 'k--')
    # plt.plot(band['xcoords'],band['eigenvalues'][:,0] - EF,'k--',label='GPAW')

    for ii in high_sym_kpoints:
        plt.axvline(ii, color='gray', lw=1, ls='--')
    plt.tick_params(direction='in')
    plt.ylim(-2, 35)
    plt.xlim(xlist.min(), xlist.max())
    plt.ylabel('E - EF (eV)', fontsize=12)
    plt.yticks(fontsize=12)
    plt.xticks(high_sym_kpoints, labels, fontsize=12)
    plt.show()



# initial rotate H or S func.
FCC(a=5.43)
  Variant name: FCC
  Special point names: GKLUWX
  Default path: GXWKGLUWLK,UX

  Special point coordinates:
    G   0.0000  0.0000  0.0000
    K   0.3750  0.3750  0.7500
    L   0.5000  0.5000  0.5000
    U   0.6250  0.2500  0.6250
    W   0.5000  0.2500  0.7500
    X   0.5000  0.0000  0.5000



UnboundLocalError: local variable 'x' referenced before assignment